In [1]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE OverloadedLists #-}

In [2]:
import Network.HTTP.Req
import qualified Data.ByteString as B
import Control.Lens hiding ((.=))
import Data.Default.Class
import Data.Aeson
import Data.Aeson.Lens
import Data.Aeson.Text
import Data.Attoparsec.Text
import Data.Monoid ((<>), mconcat)
import qualified Data.Text as T
import qualified Data.Text.IO as T
import Data.List (nub)

email <- B.init <$> B.readFile "./email"
apiKey <- B.init <$> B.readFile "./api-key"

emailT <- T.pack . init <$> readFile "./email"

narrow :: Value
narrow = Array
    [ object ["operator" .= String "sender", "operand" .= String emailT]
    , object ["operator" .= String "stream", "operand" .= String "music"]
    ]

params an
    =  "anchor"     =: an
    <> "num_before" =: 0
    <> "num_after"  =: 40
    <> "narrow"     =: encodeToLazyText narrow

r an = req GET (https "recurse.zulipchat.com" /: "api" /: "v1" /: "messages") NoReqBody jsonResponse ((basicAuth email apiKey) <> params an)

res <- responseBody <$> runReq def (r 0) :: IO Value

In [3]:
contents = res ^.. key "messages" . values . key "content" . _String

In [4]:
parseYouTube = do
    pre <- string "\"https://www.youtube.com/watch?v="
    vid <- manyTill' anyChar (char '"')
    pure $ pre <> T.pack vid <> T.pack ['"']

parseLink = do
    manyTill' anyChar (string "<a href=")
    parseYouTube

Right links = concat <$> traverse (parseOnly (nub <$> many' parseLink)) contents

In [5]:
mapM_ T.putStrLn links

"https://www.youtube.com/watch?v=O6ZNcIJmn5I"
"https://www.youtube.com/watch?v=-gBbqXgTakA"
"https://www.youtube.com/watch?v=lrSpyokJiZU"
"https://www.youtube.com/watch?v=zpKciMlWid0"
"https://www.youtube.com/watch?v=Om_SnnIgvms"
"https://www.youtube.com/watch?v=wwuoI4Vo6E4"
"https://www.youtube.com/watch?v=u47LZJIKPkU"
"https://www.youtube.com/watch?v=JkfvlQ75WUg"
"https://www.youtube.com/watch?v=Ph5psMQoirU"
"https://www.youtube.com/watch?v=4KLDlMtxt4A"
"https://www.youtube.com/watch?v=1iXKuPsfzt8"
"https://www.youtube.com/watch?v=phPbZ_QsZgk"
"https://www.youtube.com/watch?v=uAwbKHSRbGA"
"https://www.youtube.com/watch?v=Djgwyy821SI"
"https://www.youtube.com/watch?v=LzX3ijcltkA"
"https://www.youtube.com/watch?v=qqHiQNL1kSk"
"https://www.youtube.com/watch?v=arEgV6_Oah0"
"https://www.youtube.com/watch?v=JI6MLcJb_m4"
"https://www.youtube.com/watch?v=QFpTEcHy0Yc"
"https://www.youtube.com/watch?v=mi9tWkLDd2c"
"https://www.youtube.com/watch?v=YYVcRtPDoCs"
"https://www.youtube.com/watch?v=W

In [6]:
latest = last $ res ^.. key "messages" . values . key "id" . _Integer

In [7]:
res' <- responseBody <$> runReq def (r (fromIntegral latest + 1)) :: IO Value

In [8]:
contents' = res' ^.. key "messages" . values . key "content" . _String
Right links' = concat <$> traverse (parseOnly ( nub <$> many' parseLink)) contents'
mapM_ T.putStrLn links'

"https://www.youtube.com/watch?v=xGBp7pPoyz0"
"https://www.youtube.com/watch?v=lxLrTbQ2MSo"
"https://www.youtube.com/watch?v=W6FSAP5Gcvs"
"https://www.youtube.com/watch?v=_AS1qChobBA"
"https://www.youtube.com/watch?v=8tebXGHIeCo"
"https://www.youtube.com/watch?v=ya9v7jIQQ_U"
"https://www.youtube.com/watch?v=HnFJKYOgSgY"
"https://www.youtube.com/watch?v=Q1Wo1MW8IT8"
"https://www.youtube.com/watch?v=KjBvrubO9nI"
"https://www.youtube.com/watch?v=iHpHgR7pX_Y"
"https://www.youtube.com/watch?v=qJdMjRHRLfg"
"https://www.youtube.com/watch?v=unBfb2syL18"
"https://www.youtube.com/watch?v=Yuz_8Vxlyjw"
"https://www.youtube.com/watch?v=n3HIS4gZsKc"
"https://www.youtube.com/watch?v=1fHMiE2SpZ8"
"https://www.youtube.com/watch?v=ex9iMQynXaU"
"https://www.youtube.com/watch?v=Gs069dndIYk"
"https://www.youtube.com/watch?v=lUTesjKAdmc"
"https://www.youtube.com/watch?v=mdnmCBGj7qo&amp;feature=youtu.be"
"https://www.youtube.com/watch?v=B2Y7ESqe6lM"
"https://www.youtube.com/watch?v=BhUrFhI6yYw"